In [10]:
import pandas as pd
import odo
from collections import OrderedDict
from datashape import Record
from sqlalchemy import *

In [11]:
def prepare_for_copy(table_name, frame, metadata):
    shape = odo.discover(frame)
    od = OrderedDict(shape.measure.fields)
    for key in od:
        type_name = str(od[key])
        if not type_name.startswith('?'):
            od[key] = odo.dshape('?{}'.format(type_name)) 
    target_shape = odo.dshape([shape.parameters[0], Record(od)])
    target_table = odo.backends.sql.dshape_to_table(table_name, target_shape, metadata=metadata)
    return (target_shape, target_table)

In [12]:
engine = create_engine('sqlite:///local-data/budge.sqlite')
metadata = MetaData(bind=engine)

In [13]:
ec2 = pd.read_csv('/Users/ashapoch/Temp/aws-offers/AmazonEC2.csv', low_memory=False)

In [14]:
target_shape, target_table = prepare_for_copy('AmazonEC2', ec2, metadata)

In [15]:
odo.odo(ec2, target_table, dshape=target_shape)

Table('AmazonEC2', MetaData(bind=Engine(sqlite:///local-data/budge.sqlite)), Column('SKU', Text(), table=<AmazonEC2>), Column('OfferTermCode', Text(), table=<AmazonEC2>), Column('RateCode', Text(), table=<AmazonEC2>), Column('TermType', Text(), table=<AmazonEC2>), Column('PriceDescription', Text(), table=<AmazonEC2>), Column('EffectiveDate', Text(), table=<AmazonEC2>), Column('StartingRange', FLOAT(), table=<AmazonEC2>), Column('EndingRange', FLOAT(), table=<AmazonEC2>), Column('Unit', Text(), table=<AmazonEC2>), Column('PricePerUnit', FLOAT(), table=<AmazonEC2>), Column('Currency', Text(), table=<AmazonEC2>), Column('LeaseContractLength', Text(), table=<AmazonEC2>), Column('PurchaseOption', Text(), table=<AmazonEC2>), Column('OfferingClass', Text(), table=<AmazonEC2>), Column('Product_Family', Text(), table=<AmazonEC2>), Column('serviceCode', Text(), table=<AmazonEC2>), Column('Location', Text(), table=<AmazonEC2>), Column('Location_Type', Text(), table=<AmazonEC2>), Column('Instance_

In [16]:
ec2_sql = pd.read_sql('AmazonEC2', 'sqlite:///local-data/budge.sqlite', 
                      coerce_float=True,
                      parse_dates={'EffectiveDate': '%Y-%m-%d'})

In [30]:
def column_values(frame):
    for column in frame.columns:
        print(column, frame[column].unique())

In [31]:
column_values(ec2_sql)

SKU ['YQHNG5NBWUE3D67S' 'EYDF9FPAH9XHZRBR' 'WQ8JS87GX5QJ6VS4' ...,
 '8N98ZZ39AXPEH5HG' 'KSH9QSRS8SVAAQXM' 'F6SJVEQRD2XCWRMK']
OfferTermCode ['4NA7Y494T4' 'Z2E3P23VKM' 'JRTCKXETXF' 'HU7G6KETJZ' '6QCMYABX3D'
 '38NPMPTW36' 'R5XV2EPZQZ' 'MZU6U2429S' 'NQ3QZPMQV9']
RateCode ['YQHNG5NBWUE3D67S.4NA7Y494T4.6YS6EN2CT7'
 'EYDF9FPAH9XHZRBR.Z2E3P23VKM.6YS6EN2CT7'
 'WQ8JS87GX5QJ6VS4.JRTCKXETXF.6YS6EN2CT7' ...,
 'PHDC7MBYXDRUFJ3W.JRTCKXETXF.6YS6EN2CT7'
 '6YGF9V4VPXM2PCWU.6QCMYABX3D.6YS6EN2CT7'
 'ZSEV6RFE6GFENUNZ.MZU6U2429S.2TG2D8R56U']
TermType ['Reserved' 'OnDemand']
PriceDescription [ 'Red Hat Enterprise Linux (Amazon VPC), m4.xlarge instance-hours used this month'
 'Windows with SQL Server Web (Amazon VPC), r4.16xlarge instance-hours used this month'
 '$5.138 per Dedicated Windows i2.4xlarge Instance Hour' ...,
 '$0.342 per Dedicated Usage SUSE c4.xlarge Instance Hour'
 '$1.089 per On Demand Windows BYOL m4.4xlarge Instance Hour'
 '$0.772 per On Demand Windows BYOL g2.2xlarge Instance Hour']
Effec

operation ['RunInstances:0010' 'RunInstances:0202' 'RunInstances:0002'
 'RunInstances:0006' 'RunInstances:0102' 'RunInstances:000g' 'RunInstances'
 'RunInstances:0800' None 'Hourly' 'LoadBalancing:Application' 'NatGateway'
 'LoadBalancing' 'Surcharge']
Comments [None 'To be made public at the time of LHR launch, 12/13']
Dedicated_EBS_Throughput ['750 Mbps' '12000 Mbps' None '2000 Mbps' '1000 Mbps' '425 Mbps'
 '1750 Mbps' '3000 Mbps' '1600 Mbps' '14000 Mbps' '450 Mbps' '3500 Mbps'
 '4000 Mbps' '10000 Mbps' '850 Mbps' '7000 Mbps' '800 Mbps' '6000 Mbps'
 '400 Mbps' '500 Mbps']
Enhanced_Networking_Supported ['Yes' None]
GPU [ nan  16.   1.   4.]
Instance_Capacity_10xlarge [ nan   1.]
Instance_Capacity_2xlarge [ nan   4.   8.   5.]
Instance_Capacity_4xlarge [ nan   2.   4.]
Instance_Capacity_8xlarge [ nan   1.   2.]
Instance_Capacity_large [ nan  32.  16.  22.]
Instance_Capacity_medium [ nan  32.]
Instance_Capacity_xlarge [ nan   8.  16.  11.]
Intel_AVX_Available [None 'Yes']
Intel_AVX2_Ava

In [29]:
compared_ec2_instances = ec2_sql[(ec2_sql.Product_Family == 'Compute Instance') & 
                                 (ec2_sql.Tenancy == 'Shared') & 
                                 (ec2_sql.Operating_System == 'Linux') &
                                 (ec2_sql.Location == 'US East (Ohio)')]


In [32]:
column_values(compared_ec2_instances)

SKU ['YQZKQQ43X7CBE4ZD' 'B7KJQVXZZNDAS23N' 'AW2T9CUGJYGQKTY8'
 '9DMUVQNHGNHC7R92' 'Q236HUQXMUFFR9AN' 'JVT8Z9JR8H2KTMEY'
 'ZWW6ZZSFWQM7H534' 'C2XHT7MUVASJ7UQ3' '9F7GGKJQGM6J387N'
 '2N2QH6UEJZ5GUPT8' 'URPTCUXU96S8XJUV' 'PBMCNRQPJ8XA43PT'
 'R4CKBWU7M6GSC7QB' '65YDGPXVA9GTBYCA' 'DBBN8V6AE5WQCCGZ'
 'GZSHSGVERG8544YU' '34ZEJZENQ3WGN6MA' 'MW8UQZATN9TSYX2K'
 'RXYEZ3QXT2GWNMRK' 'W4ZRFQ4Q6W3P22C4' 'M6S6AKBD4WYEHANV'
 'Y9Z92WYATPYKXSM2' 'GWP38ESW2CNEVPUS' 'R99FC88U7735H9RR'
 'GSMN37GEEUV2CC27' '9F2Q74QEE8HZE6A8' 'JNBD8ZZ5JSFMNNYR'
 '82N8PRZ8U7GE8XRM' '5G4M46QWK9ZC9QDT' 'RGU9HRNUAS2TX83W'
 'VU6WDT5USNCYBZEU' '9MXQF8NSPZESJJUW' '3VWERDY4UHEZUS9F'
 'RHW8EAZNVJA4KSWC' 'PSAR6KAESEV4S8JY' '93HAYG7AFZGMJKJY'
 'RWY5P9FB6JQYYM78' 'DWQDRJ9AWUJ8BZKH' 'F9GPUA3E29X6GJVE'
 'WTADZ55ETP6ZEN73' 'DJPGFVCZAKBSEZ3N' 'J3ASENPUVKBMMQG3'
 'P3R3B44DZXKTKMYR' 'ZZNXUPSQ9BSA2M88' 'MD5WDRNPRHM2XAGC']
OfferTermCode ['HU7G6KETJZ' 'NQ3QZPMQV9' 'R5XV2EPZQZ' '6QCMYABX3D' 'MZU6U2429S'
 '38NPMPTW36' '4NA7Y494T4' 'JRTCKXETXF' 'Z2E3

In [62]:
grouped_by_term = compared_ec2_instances[
    compared_ec2_instances.Instance_Type == 't2.medium'].groupby(['OfferTermCode'])

In [63]:
grouped_by_term.groups

{'38NPMPTW36': Int64Index([27134, 122307], dtype='int64'),
 '4NA7Y494T4': Int64Index([86477], dtype='int64'),
 '6QCMYABX3D': Int64Index([59396, 81993], dtype='int64'),
 'HU7G6KETJZ': Int64Index([1470, 68337], dtype='int64'),
 'JRTCKXETXF': Int64Index([75011], dtype='int64'),
 'MZU6U2429S': Int64Index([22923, 24698], dtype='int64'),
 'NQ3QZPMQV9': Int64Index([52505, 141466], dtype='int64'),
 'R5XV2EPZQZ': Int64Index([78948, 88825], dtype='int64'),
 'Z2E3P23VKM': Int64Index([146520], dtype='int64')}

In [66]:
from pprint import pprint
for g in grouped_by_term.groups:
    pprint(grouped_by_term.get_group(g)[['OfferTermCode', 'RateCode', 'TermType', 
                                         'PurchaseOption', 'LeaseContractLength', 'OfferingClass', 
                                         'PricePerUnit', 'PriceDescription']].to_dict())
    print('-' * 100)

{'LeaseContractLength': {27134: '3yr', 122307: '3yr'},
 'OfferTermCode': {27134: '38NPMPTW36', 122307: '38NPMPTW36'},
 'OfferingClass': {27134: 'standard', 122307: 'standard'},
 'PriceDescription': {27134: 'Upfront Fee',
                      122307: 'Linux/UNIX (Amazon VPC), t2.medium '
                              'instance-hours used this month'},
 'PricePerUnit': {27134: 289.0, 122307: 0.010999999999999999},
 'PurchaseOption': {27134: 'Partial Upfront', 122307: 'Partial Upfront'},
 'RateCode': {27134: '2N2QH6UEJZ5GUPT8.38NPMPTW36.2TG2D8R56U',
              122307: '2N2QH6UEJZ5GUPT8.38NPMPTW36.6YS6EN2CT7'},
 'TermType': {27134: 'Reserved', 122307: 'Reserved'}}
----------------------------------------------------------------------------------------------------
{'LeaseContractLength': {86477: '1yr'},
 'OfferTermCode': {86477: '4NA7Y494T4'},
 'OfferingClass': {86477: 'standard'},
 'PriceDescription': {86477: 'Linux/UNIX (Amazon VPC), t2.medium '
                             'instance-